In [1]:
%matplotlib tk
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import FuncFormatter
from matplotlib import dates
from datetime import datetime, timedelta
import pandas
import os
from numpy import isnan

In [16]:
def seconds_to_string(n: int, _) -> str:
    hour = int(n // 3600)
    seconds = int(n % 60)
    minutes = int((n // 60) % 60)
    return f"{hour}:{minutes:02}:{seconds:02}"

In [3]:
def secant_is_ok_func(df):
    prev_value = df['timer_value'].shift(1)
    prev_index = df.index.to_series().shift(1)
    index_diff = df.index - prev_index
    seconds_slope = (df['timer_value'] - prev_value) / (index_diff / 29.97)
    secant_is_ok = seconds_slope.apply(lambda x: x > -1.5 or isnan(x))
    return secant_is_ok



In [18]:
for fp in os.listdir('csvs'):
    df = pandas.read_csv(os.path.join('csvs', fp), index_col='frame')
    df = df[df['timer_value'] != -1]

    while True: # filter by secant lines— should not fall > 1/sec / sec (ish)
        ok = secant_is_ok_func(df)

        df = df.loc[ok & ok.shift(-1), :]

        if (~ok).sum() <= 1 and (~secant_is_ok_func(df)).sum() <= 1:
            break
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.plot(df.index/29.97, df['timer_value'], markersize=1)

    ax.xaxis.set_major_formatter(FuncFormatter(seconds_to_string))
    ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(base=3600*3))
    ax.set_xlabel('time into stream')

    ax.yaxis.set_major_formatter(FuncFormatter(seconds_to_string))
    ax.yaxis.set_major_locator(mpl.ticker.MultipleLocator(base=3600/2))
    ax.set_ylabel('time remaining')
    vod, v = fp.split('|')
    ax.set_title(f'VOD {vod}')
    fig.text(0.1, 0.9, f'Video available at youtube.com/watch?v={v[:-4]}')
    fig.text(0.6, 0.9, f'Made by u/2435191 with cv2, tesseract, pandas, and mpl')
    fig.savefig(f'graphs/{vod}.png', dpi=400)
